# Setup

Using Colab

## Installs

In [ ]:
!pip install -U cellxgene-census
!pip install "git+https://github.com/kmaherx/ScBMLP.git"

  Cloning https://github.com/kmaherx/ScBMLP.git to /tmp/pip-req-build-9xthg45n
  Running command git clone --filter=blob:none --quiet https://github.com/kmaherx/ScBMLP.git /tmp/pip-req-build-9xthg45n
  Resolved https://github.com/kmaherx/ScBMLP.git to commit 5125961f1b9e5f6ac33f8167dfbca6d2a66aa218
  Resolved https://github.com/kmaherx/ScBMLP.git to commit 5125961f1b9e5f6ac33f8167dfbca6d2a66aa218
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Installi

## Imports

In [2]:
# from scripts.datasets import census_classes

import scanpy as sc
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"

ModuleNotFoundError: No module named 'scanpy'

In [ ]:
# import cellxgene_census
# census_version = "2025-01-30"

# Data

In [ ]:
census_config = {
    "organism" : "Homo sapiens",
    "var_value_filter" : "feature_type in ['protein_coding']",
    "obs_value_filter" : "sex in ['male', 'female'] and cell_type == 'hepatocyte' and disease == 'normal'",
    "var_column_names" : ["feature_id", "feature_name", "feature_type", "feature_length"],
    "obs_column_names" : ["cell_type", "sex", "assay", "suspension_type"],
}
class_key = "sex"

In [ ]:
# with cellxgene_census.open_soma(census_version=census_version) as census:
#     adata = cellxgene_census.get_anndata(
#         census=census,
#         organism=census_config["organism"],
#         var_value_filter=census_config["var_value_filter"],
#         obs_value_filter=census_config["obs_value_filter"],
#         var_column_names=census_config["var_column_names"],
#         obs_column_names=census_config["obs_column_names"],
#     )

In [ ]:
adata = sc.read_h5ad("/content/drive/MyDrive/census_test_data.h5ad")

In [ ]:
adata

AnnData object with n_obs × n_vars = 78776 × 20045
    obs: 'cell_type', 'sex', 'assay', 'suspension_type', 'disease'
    var: 'feature_id', 'feature_name', 'feature_type', 'feature_length'

In [3]:
total_counts = adata.X.toarray().sum(axis=1)
px.histogram(
    x=total_counts,
)

NameError: name 'adata' is not defined

In [ ]:
min_counts_per_cell = 100
max_counts_per_cell = 50_000
sc.pp.filter_cells(adata, min_counts=min_counts_per_cell)
sc.pp.filter_cells(adata, max_counts=max_counts_per_cell)

NameError: name 'sc' is not defined

In [4]:
adata

NameError: name 'adata' is not defined

In [11]:
total_counts = adata.X.toarray().sum(axis=1)
px.histogram(
    x=total_counts,
)

In [12]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [13]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000)
adata.raw = adata
adata = adata[:, adata.var.highly_variable]

In [14]:
adata

View of AnnData object with n_obs × n_vars = 78000 × 5000
    obs: 'cell_type', 'sex', 'assay', 'suspension_type', 'disease', 'n_counts'
    var: 'feature_id', 'feature_name', 'feature_type', 'feature_length', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [15]:
sc.pp.pca(adata, n_comps=5)

/usr/local/lib/python3.12/dist-packages/scanpy/preprocessing/_pca/__init__.py:385: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.



In [16]:
px.scatter(
    x=adata.obsm["X_pca"][:, 1],
    y=adata.obsm["X_pca"][:, 2],
    color=adata.obs["sex"],
    title="PCA of Gene Expression Data",
    labels={"x": "PC1", "y": "PC2"},
    width=600,
    height=600,
)

In [17]:
!pip install git+https://github.com/kmaherx/ScBMLP.git

  Cloning https://github.com/kmaherx/ScBMLP.git to /tmp/pip-req-build-gmilc1yl
  Running command git clone --filter=blob:none --quiet https://github.com/kmaherx/ScBMLP.git /tmp/pip-req-build-gmilc1yl
  Resolved https://github.com/kmaherx/ScBMLP.git to commit 37e73b935b2c66b1107e7fa1f05548a7e1b85183
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from scripts.datasets import get_split_idxs, get_type_datasets

val_split = 0.15
random_state = 0
DEVICE = "cuda"

train_indices, val_indices, test_indices = get_split_idxs(
    adata, val_split=val_split, random_state=random_state,
)
# use cpu for dataset creation to avoid sparse cuda issues, model will move to gpu
train_dataset, val_dataset, test_dataset = get_type_datasets(
    adata, train_indices, val_indices, test_indices, class_key, device="cpu",
)

# Train

In [19]:
from scripts.bmlp import ScBMLPClassifier, Config

In [20]:
d_hidden = 64
n_epochs = 100
lr = 1e-4
DEVICE = "cuda"

n_cells, n_genes = adata.shape
class_key = "sex"
n_classes = adata.obs[class_key].nunique()

n_epochs = 100

In [21]:
cfg = Config(
    d_input=n_genes,
    d_hidden=d_hidden,
    d_output=n_classes,
    n_epochs=n_epochs,
    lr=lr,
    device=DEVICE,
)
model = ScBMLPClassifier(cfg)
train_losses, val_losses = model.fit(train_dataset, val_dataset)

Training for 100 epochs:   0%|          | 0/100 [01:33<?, ?it/s]


NotImplementedError: Could not run 'aten::as_strided' with arguments from the 'SparseCUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::as_strided' is only available for these backends: [CPU, CUDA, Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradMAIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastMTIA, AutocastMAIA, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor

In [ ]:
# Combine train and val losses into a single plot
loss_df = pd.DataFrame({
    'Epoch': list(range(len(train_losses))) + list(range(len(val_losses))),
    'Loss': train_losses + val_losses,
    'Type': ['Train'] * len(train_losses) + ['Validation'] * len(val_losses)
})

px.line(loss_df, x='Epoch', y='Loss', color='Type', 
        title='Training and Validation Loss', 
        labels={'Loss': 'Loss', 'Epoch': 'Epoch'}).show()

In [ ]:
train_dataset[0][0].shape

torch.Size([20045])